In [1]:
!nvidia-smi

Sun Apr 23 17:10:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0    45W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# # This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# # Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
# !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
# !python rapidsai-csp-utils/colab/env-check.py

In [4]:
# # This will update the Colab environment and restart the kernel.  Don't run the next cell until you see the session crash.
# !bash rapidsai-csp-utils/colab/update_gcc.sh
# import os
# os._exit(00)

In [5]:
# # This will install CondaColab.  This will restart your kernel one last time.  Run this cell by itself and only run the next cell once you see the session crash.
# import condacolab
# condacolab.install()

In [6]:
# # you can now run the rest of the cells as normal
# import condacolab
# condacolab.check()

In [7]:
# # Installing RAPIDS is now 'python rapidsai-csp-utils/colab/install_rapids.py <release> <packages>'
# # The <release> options are 'stable' and 'nightly'.  Leaving it blank or adding any other words will default to stable.
# !python rapidsai-csp-utils/colab/install_rapids.py stable
# import os
# os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
# os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
# os.environ['CONDA_PREFIX'] = '/usr/local'

In [8]:
import cudf, cuml, cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors
from cuml.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
import gc
import math
import re
from cuml.cluster import KMeans
from cuml.ensemble import RandomForestClassifier as cuRFC
import pandas as pd
print('RAPIDS',cuml.__version__)

RAPIDS 22.12.00


In [9]:
# ====================================================
# Data Loading
# ====================================================
train_df = pd.read_csv('/content/drive/MyDrive/AMC 2023/train.csv')
train_df = train_df.dropna(subset = ["TITLE"]).reset_index(drop = True)
test_df = pd.read_csv('/content/drive/MyDrive/AMC 2023/test.csv')

print(f"train.shape: {train_df.shape}")
display(train_df.head())
print(f"test.shape: {test_df.shape}")
display(test_df.head())

train.shape: (2249686, 9)


,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH,title_cluster_bert,brand_cluster_bert,bullet_cluster_bert
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,NaN,1650,2125.980000,1772,7782,4511
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",NaN,2755,393.700000,1052,146,2189
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495,17,4706,4113
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574,1933,736,7176
4,283658,The United Empire Loyalists: A Chronicle of th...,NaN,NaN,6112,598.424000,541,2049,3


test.shape: (734736, 8)


,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,title_cluster_bert,brand_cluster_bert,bullet_cluster_bert
0,604373,Manuel d'Héliogravure Et de Photogravure En Re...,NaN,NaN,6142,869,843,3
1,1729783,DCGARING Microfiber Throw Blanket Warm Fuzzy P...,[QUALITY GUARANTEED: Luxury cozy plush polyest...,<b>DCGARING Throw Blanket</b><br><br> <b>Size ...,1622,1478,5655,2588
2,1871949,I-Match Auto Parts Front License Plate Bracket...,"[Front License Plate Bracket Made Of Plastic,D...",Replacement for The Following Vehicles:2020 LE...,7540,1370,3233,8957
3,1107571,PinMart Gold Plated Excellence in Service 1 Ye...,[Available as a single item or bulk packed. Se...,Our Excellence in Service Lapel Pins feature a...,12442,141,5457,7413
4,624253,"Visual Mathematics, Illustrated by the TI-92 a...",NaN,NaN,6318,755,4594,3


In [10]:
train = cupy.load("/content/drive/MyDrive/AMC 2023/mdeberta-v3-base-embeddings-brand.npz")["embeddings"]
test = cupy.load("/content/drive/MyDrive/AMC 2023/mdeberta-v3-base-embeddings_test.npz")["embeddings"]

In [11]:
kmeans = KMeans(n_clusters=10000, max_iter=500, init='scalable-k-means++')

In [12]:
%time
kmeans.fit(train)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.2 µs


KMeans()

In [13]:
train_df["bullet_cluster_mdeberta"] = cupy.asnumpy(kmeans.predict(train)).astype(int)
test_df["bullet_cluster_mdeberta"] = cupy.asnumpy(kmeans.predict(test)).astype(int)

In [14]:
train_df.to_csv("/content/drive/MyDrive/AMC 2023/train.csv", index = None)
test_df.to_csv("/content/drive/MyDrive/AMC 2023/test.csv", index = None)

In [15]:
display(train_df.head())

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH,title_cluster_bert,brand_cluster_bert,bullet_cluster_bert,bullet_cluster_mdeberta
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,NaN,1650,2125.980000,1772,7782,4511,3834
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",NaN,2755,393.700000,1052,146,2189,1170
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495,17,4706,4113,6015
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574,1933,736,7176,3848
4,283658,The United Empire Loyalists: A Chronicle of th...,NaN,NaN,6112,598.424000,541,2049,3,7944
